In [1]:
f1 = open('user_5k_avg', 'r').read()
user_avg = eval(f1)

f2 = open('business_5k_avg', 'r').read()
business_avg = eval(f2)

f3 = open('review_5k_user', 'r').read()
review5k_user = eval(f3)

f4 = open('review_5k_business', 'r').read()
review5k_business = eval(f4)

f5 = open('review_5k_rating', 'r').read()
review5k_rating = eval(f5)

f6 = open('review_5k_text', 'r').read()
review5k_text = eval(f6)

f7 = open('relation_5k', 'r').read()
relation = eval(f7)


In [2]:
import random 
import math
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF
from sklearn.utils.extmath import randomized_svd
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer


<h1>Parameters</h1>

In [77]:
random_test = random.sample(xrange(len(review5k_rating)), 1000)

train_user = []
train_business = []
train_rating = []
train_text = []

test_user = []
test_business = []
test_rating = []

for r in xrange(len(review5k_rating)):
    if r in random_test:
        test_user.append(review5k_user[r])
        test_business.append(review5k_business[r])
        test_rating.append(review5k_rating[r])
    else:
        train_user.append(review5k_user[r])
        train_business.append(review5k_business[r])
        train_rating.append(review5k_rating[r])
        train_text.append(review5k_text[r])
        
K_topic = 10
Times = 50
DocWord = 300
DocTopic = 5

sim_val = 0.85 # default = 0.85
corpus = 0.8

<h1>Initialization</h1>

In [4]:
num_user = len(user_avg)
num_business = len(business_avg)
num_train = len(train_rating)
num_test = len(test_rating)

mu = np.mean(train_rating)

def float2int(f):
    if f < 1.5: return 1
    if f < 2.5: return 2
    if f < 3.5: return 3
    if f < 4.5: return 4
    return 5

tokenizer = RegexpTokenizer(r'\w+')
en_stop = get_stop_words('en')
p_stemmer = PorterStemmer()

def prep(doc):
    raw = doc.lower().replace("\n", "").replace("\t", "")
    tokens = tokenizer.tokenize(raw)
    stopped_tokens = [i for i in tokens if not i in en_stop]
    texts = [p_stemmer.stem(i) for i in stopped_tokens]
    return (" ").join(texts)

from sklearn import linear_model
def MLR(X, Y):
    reg = linear_model.LinearRegression()
    reg.fit(X, Y)
    return reg.coef_

In [5]:
row = np.array([])
col = np.array([])
val = np.array([])

# user-business rating matrix
Rij = csr_matrix((val,(row,col)), shape=(num_user,num_business)).toarray()
for r in xrange(num_train):
    Rij[train_user[r]][train_business[r]] = train_rating[r]

# user-user relation matrix
Tij = csr_matrix((val,(row,col)), shape=(num_user,num_user)).toarray()
for u in relation:
    for f in u[1]:
        Tij[u[0]][f] = 1
 

<h1>No Model</h1>

In [6]:
Ubb = []
for i in xrange(num_user):
    Ubb.append([])
    for j in xrange(num_business):
        val = user_avg[i] + business_avg[j] - mu
        if val < 1: val = 1
        if val > 5: val = 5
        Ubb[i].append(val) 

In [7]:
UbbPd = []
for r in xrange(num_test):
    UbbPd.append(Ubb[test_user[r]][test_business[r]]) 

UbbPdInt = []
for r in UbbPd: 
    UbbPdInt.append(float2int(r)) 

Ubb_rmse = mean_squared_error(test_rating, UbbPd)  
UbbPdInt_rmse = mean_squared_error(test_rating, UbbPdInt) 

print Ubb_rmse
print UbbPdInt_rmse

0.897182401946
0.982


<h1>Basic Model</h1>

In [ ]:
BasicIn = []
for i in xrange(num_user):
    BasicIn.append([])
    for j in xrange(num_business):
        val = user_avg[i] + business_avg[j] - mu
        if val < 1: val = 1
        if val > 5: val = 5
        BasicIn[i].append(val) 
        
for r in xrange(num_train):
    BasicIn[train_user[r]][train_business[r]] = train_rating[r]


In [ ]:
model = NMF(n_components=K_topic, init='random', random_state=0)
U = model.fit_transform(BasicIn);
V = model.components_;
BasicOut = np.dot(U,V)

In [ ]:
BasicPd = []
for r in xrange(num_test):
    BasicPd.append(BasicOut[test_user[r]][test_business[r]]) 

BasicPdInt = []
for r in BasicPd: 
    BasicPdInt.append(float2int(r)) 

BasicPd_rmse = mean_squared_error(test_rating, BasicPd)  
BasicPdInt_rmse = mean_squared_error(test_rating, BasicPdInt) 

print BasicPd_rmse
print BasicPdInt_rmse

<h1>Topic Model</h1>

In [78]:
Breview = [""]*num_business
for r in xrange(num_train):
    Breview[train_business[r]] += prep(train_text[r])

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=DocWord, stop_words='english')
tf = tf_vectorizer.fit_transform(Breview)

lda = LatentDirichletAllocation(n_topics=DocTopic, max_iter=5, learning_method='online',learning_offset=50.,random_state=0)
DocTopDist = lda.fit_transform(tf)


In [21]:
row = np.array([])
col = np.array([])
val = np.array([])

# delta -> difference between rating and ubb [u*b], delta2 -> rating boolean
delta = csr_matrix((val,(row,col)), shape=(num_user,num_business)).toarray()
delta2 = csr_matrix((val,(row,col)), shape=(num_user,num_business)).toarray()
for r in xrange(num_train):
    delta[train_user[r]][train_business[r]] = train_rating[r] - Ubb[train_user[r]][train_business[r]]
    delta2[train_user[r]][train_business[r]] = 1


In [100]:
deltaReg = []
for i in xrange(num_user):
    X = [[0,0,0,0,0], [1,0,0,0,0], [0,1,0,0,0], [0,0,1,0,0], [0,0,0,1,0], [0,0,0,0,1]]; Y = [0,0,0,0,0,0]
    for j in xrange(num_business):
        if delta2[i][j]:
            X.append(list(DocTopDist[j]))
            Y.append(delta[i][j])
    coef = MLR(X, Y)
    deltaReg.append(np.dot(DocTopDistTransf, coef))
    

In [103]:
row = np.array([])
col = np.array([])
val = np.array([])
TopicIn = csr_matrix((val,(row,col)), shape=(num_user,num_business)).toarray()
for i in xrange(num_user):
    for j in xrange(num_business):
        val = Ubb[i][j] + math.atan(deltaReg[i][j])
        if val < 1: val = 1
        if val > 5: val = 5
        TopicIn[i][j] = val

TopicInPd = []
for r in xrange(num_test):
    TopicInPd.append(TopicIn[test_user[r]][test_business[r]]) 

TopicInPd_rmse = mean_squared_error(test_rating, TopicInPd)  
print TopicInPd_rmse

0.835722422368


<h2> TopicMF Option 2

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,max_features=K_topic,stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(Breview)

nmf = NMF(n_components=K_topic, random_state=1,alpha=.1, l1_ratio=.5)
DocTopicDist2 = nmf.fit_transform(tfidf)


In [ ]:
DocCosine_norm = []
for n in xrange(num_business):
    DocCosine_norm.append(math.sqrt(np.dot(DocTopicDist2[n], DocTopicDist2[n])))

for n in xrange(num_business):
    if DocCosine_norm[n]:
        DocTopicDist2[n] = 1.0*DocTopicDist2[n]/DocCosine_norm[n]
    else: 
        DocTopicDist2[n] = 0
    
DocSim = np.dot(DocTopicDist2, DocTopicDist2.T)

In [ ]:
DocSim

In [ ]:
row = np.array([])
col = np.array([])
val = np.array([])

# tempR1 -> difference between rating and ubb [u*b], tempR2 -> rating boolean
tempR1 = csr_matrix((val,(row,col)), shape=(num_user,num_business)).toarray()
tempR2 = csr_matrix((val,(row,col)), shape=(num_user,num_business)).toarray()
for r in xrange(num_train):
    tempR1[train_user[r]][train_business[r]] = train_rating[r] - Ubb[train_user[r]][train_business[r]]
    tempR2[train_user[r]][train_business[r]] = 1

tempR3 = csr_matrix((val,(row,col)), shape=(num_business,num_business)).toarray()
tempR4 = csr_matrix((val,(row,col)), shape=(num_business,num_business)).toarray()
for i in xrange(num_business):
    for j in xrange(num_business):
        if DocSim[i][j] > sim_val:
            tempR4[i][j] = sim_val
            tempR3[i][j] = 1

tempR4 = np.dot(tempR1, tempR4)
tempR5 = np.dot(tempR2, tempR3)

tempR6 = csr_matrix((val,(row,col)), shape=(num_user,num_business)).toarray()
for i in xrange(num_user):
    for j in xrange(num_business):
        if tempR5[i][j] > 0:
            tempR6[i][j] = tempR4[i][j]/tempR5[i][j]


In [ ]:
row = np.array([])
col = np.array([])
val = np.array([])
TopicIn2 = csr_matrix((val,(row,col)), shape=(num_user,num_business)).toarray()
for i in xrange(num_user):
    for j in xrange(num_business):
        val = Ubb[i][j] + tempR6[i][j]
        if val < 1: val = 1
        if val > 5: val = 5
        TopicIn2[i][j] = val

TopicIn2Pd = []
for r in xrange(num_test):
    TopicIn2Pd.append(TopicIn2[test_user[r]][test_business[r]]) 

TopicInPd_rmse2 = mean_squared_error(test_rating, TopicIn2Pd)  
print TopicInPd_rmse2

<h1> Social Model</h1>

In [104]:
import networkx as nx

node_list = []
for u in xrange(num_user):
    node_list.append(u)

G=nx.Graph()
G.add_nodes_from(node_list)
for u in relation:
    for f in u[1]:
        G.add_edge(u[0], f)

PR = nx.pagerank(G)


In [105]:
from operator import itemgetter
sorted_PR = sorted(PR.items(), key=itemgetter(1), reverse=True)

rank = {}
for u in xrange(num_user):
    rank[sorted_PR[u][0]] = u

rev_rank = []
for u in xrange(num_user):
    rev_rank.append(rank[u]+1)

Uweight = []
for ri in rev_rank:
    Uweight.append(1.0/(1.0+ math.log(ri)))    

<h1> TEMP

In [ ]:
278 + 2071 + 766 + 429 + 246 + 178

In [ ]:
for i in xrange(num_test):
    print test_rating[i], UbbPd[i], tempR6, TopicIn2Pd[i]

In [90]:
from collections import Counter
Counter(review5k_user)

Counter({0: 1,
         1: 47,
         2: 11,
         3: 51,
         4: 1,
         5: 17,
         7: 13,
         8: 1,
         9: 2,
         10: 1,
         11: 2,
         12: 2,
         13: 16,
         14: 1,
         15: 5,
         16: 1,
         17: 1,
         18: 3,
         19: 1,
         20: 2,
         21: 4,
         22: 2,
         23: 1,
         24: 3,
         25: 1,
         27: 1,
         28: 2,
         29: 1,
         30: 1,
         31: 1,
         32: 1,
         33: 4,
         35: 2,
         36: 1,
         37: 2,
         38: 24,
         39: 30,
         40: 53,
         41: 51,
         42: 49,
         43: 61,
         44: 61,
         45: 29,
         46: 6,
         47: 2,
         48: 11,
         49: 1,
         50: 11,
         51: 2,
         52: 5,
         53: 2,
         54: 56,
         55: 2,
         56: 2,
         57: 3,
         58: 3,
         59: 1,
         60: 1,
         61: 1,
         62: 1,
         63: 1,
         64: 1,


In [91]:
from collections import Counter
Counter(review5k_business)

Counter({0: 4,
         2: 12,
         3: 7,
         4: 3,
         5: 8,
         6: 7,
         7: 3,
         8: 6,
         9: 2,
         10: 2,
         11: 11,
         12: 6,
         13: 8,
         15: 8,
         16: 1,
         17: 10,
         18: 3,
         19: 1,
         20: 10,
         21: 8,
         23: 1,
         25: 7,
         26: 13,
         27: 12,
         28: 12,
         29: 7,
         30: 12,
         31: 2,
         32: 9,
         33: 3,
         34: 1,
         35: 4,
         36: 2,
         37: 2,
         38: 3,
         39: 6,
         40: 3,
         41: 6,
         42: 9,
         43: 4,
         44: 1,
         46: 6,
         47: 12,
         48: 1,
         49: 11,
         50: 3,
         51: 8,
         52: 4,
         53: 1,
         54: 5,
         55: 1,
         57: 3,
         58: 6,
         59: 13,
         60: 7,
         61: 15,
         62: 6,
         63: 7,
         64: 1,
         65: 6,
         66: 9,
         67: 2,
     